In [115]:
import numpy as np
import pandas as pd
import jieba
import matplotlib.pyplot as plt
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from joblib import dump, load
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pickle
import math

In [2]:
data = pd.read_csv('dataset.csv')

In [3]:
data.shape

(32526, 3)

In [4]:
data.head()

,qid,question,answer
0,1,补发网银盾,您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。
1,2,代发工资,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。
2,3,对帐单查询打印,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。
3,4,理财产品取号,请说出您需要办理的业务
4,5,密码修改,请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。


In [70]:
data.loc[0, 'question']

'补发网银盾'

In [68]:
data

,qid,question,answer
0,1,补发网银盾,您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。
1,2,代发工资,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。
2,3,对帐单查询打印,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。
3,4,理财产品取号,请说出您需要办理的业务
4,5,密码修改,请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。
...,...,...,...
32521,69115,我要点咖啡,请选择您要的口味
32522,69116,帮忙点咖啡,请选择您要的口味
32523,69117,问题1,测试
32524,69118,1111,johlhk


In [5]:
# 加载语料
with open('stopwords.txt') as file:
    stopwords = file.read().split('\n')
    
corpus = []

for question in data['question']:
    question = str(question).strip()
    corpus.append(" ".join(x for x in jieba.cut(question) if x not in stopwords))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/k2/dnh2dnfx6937855thyp78q9r0000gn/T/jieba.cache
Loading model cost 0.844 seconds.
Prefix dict has been built successfully.


In [6]:
corpus

['补发 网银盾',
 '代发 工资',
 '帐单 查询 打印',
 '理财产品 取号',
 '密码 修改',
 '查询 名下 所有 账户',
 '交易 流水 明细 证明',
 '个人 结售汇',
 '现金 取款',
 '账户 销户',
 '转账 业务',
 '现金 存款',
 '贵金属 业务',
 '个人账户 管理',
 '开 特色 卡',
 '吞卡 证明',
 '补打 发票',
 '查询 取款',
 '本卡 转 本卡',
 '本卡 转 现金',
 '5 万 以内 转账',
 '5 万 20 万 转账',
 '20 万 200 万 转账',
 '200 万 以上 转账',
 '结售汇 不取 现金',
 '结售汇 及取 现金',
 '银星 收款',
 '仅 查询',
 '银星 汇款',
 '西联 收款',
 '西联 汇款',
 '身份证 开卡',
 '证件 开卡',
 '信用卡 非现金 还款',
 '信用卡 业务',
 '建信 保险',
 '保险',
 '社保 业务',
 '存单 业务',
 '查询 外汇 汇率',
 '存折 解挂',
 '纪念币 业务',
 '开 普通 龙卡',
 '结算 通卡 业务',
 '换 手机 银行 手机号',
 '粤 通卡 充值',
 '充值 缴费',
 '柜台 缴费',
 '信用卡 短信 业务',
 '出国 惠 增值 服务',
 '咨询 外币 旅行支票 光票 托收 业务',
 '保费',
 '卡折 挂失',
 '通知 存款 转 活期 咨询',
 '手机 银行 通知 存款 转 活期',
 '网银 首次 购买 理财 风险 测评 提示',
 '信用卡 挂失 生效 时间',
 'II 类 账户 定义',
 '手机 银行 活期 转 定期',
 '私人 银行 客户 助业贷 扩大 抵押物 范围',
 '外币 储蓄 利率',
 '蓝豹 百大 龙卡 联名卡 种类',
 '咨询 汇率 牌价',
 '短信 推送 业务 类型 客户 反感',
 '网银 支付 方式 适用 对象',
 '信用卡 附属 卡开卡 条件',
 '是否 信用卡 账单 日前 交易 都 会 反映 一期 账单 上',
 '外币 储蓄 定期 转存',
 '保管箱 销户',
 '下载 个人 金交所 交易 客户端',
 '私享品 陶瓷',
 '查询 两年 前 交易 明细',
 '活期存款 账户 销户'

In [93]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus)
kmeans = KMeans(n_clusters=1000).fit(tfidf)


# 打印出各个类的中心点
# print(kmeans.cluster_centers_)
# for index, label in enumerate(kmeans.labels_, 1):
#     print("index: {}, label: {}".format(index, label))

# 样本距其最近的聚类中心的平方距离之和inertia_，用来评估分类的准确度，值越小越好
# 调整超参数n_clusters可以改善inertia_
print("inertia: {}".format(kmeans.inertia_))

dump(kmeans, 'kmeans_1000.joblib')

inertia: 18842.383989936447


['kmeans_1000.joblib']

In [94]:
dump(tfidf_vectorizer, 'tfidf_vectorizer_1000.joblib')

['tfidf_vectorizer_1000.joblib']

In [163]:
kmeans = load('kmeans_1000.joblib')

In [164]:
tfidf_vectorizer = load('tfidf_vectorizer_1000.joblib')

In [165]:
user_input = ['补发 网银盾']
result = kmeans.predict(tfidf_vectorizer.transform(user_input))

In [166]:
result[0]

3

In [140]:
for index, label in enumerate(kmeans.labels_):
    print("index: {}, label: {}".format(index, label))

index: 0, label: 3
index: 1, label: 3
index: 2, label: 537
index: 3, label: 674
index: 4, label: 264
index: 5, label: 343
index: 6, label: 81
index: 7, label: 714
index: 8, label: 440
index: 9, label: 699
index: 10, label: 898
index: 11, label: 440
index: 12, label: 749
index: 13, label: 3
index: 14, label: 3
index: 15, label: 251
index: 16, label: 3
index: 17, label: 399
index: 18, label: 3
index: 19, label: 440
index: 20, label: 60
index: 21, label: 60
index: 22, label: 60
index: 23, label: 60
index: 24, label: 456
index: 25, label: 456
index: 26, label: 368
index: 27, label: 18
index: 28, label: 382
index: 29, label: 368
index: 30, label: 382
index: 31, label: 322
index: 32, label: 322
index: 33, label: 179
index: 34, label: 898
index: 35, label: 401
index: 36, label: 401
index: 37, label: 898
index: 38, label: 542
index: 39, label: 429
index: 40, label: 3
index: 41, label: 898
index: 42, label: 334
index: 43, label: 385
index: 44, label: 232
index: 45, label: 949
index: 46, label: 

index: 1692, label: 136
index: 1693, label: 3
index: 1694, label: 307
index: 1695, label: 569
index: 1696, label: 3
index: 1697, label: 3
index: 1698, label: 3
index: 1699, label: 3
index: 1700, label: 3
index: 1701, label: 119
index: 1702, label: 3
index: 1703, label: 70
index: 1704, label: 252
index: 1705, label: 252
index: 1706, label: 3
index: 1707, label: 3
index: 1708, label: 3
index: 1709, label: 65
index: 1710, label: 423
index: 1711, label: 3
index: 1712, label: 3
index: 1713, label: 638
index: 1714, label: 115
index: 1715, label: 392
index: 1716, label: 3
index: 1717, label: 361
index: 1718, label: 3
index: 1719, label: 3
index: 1720, label: 3
index: 1721, label: 3
index: 1722, label: 3
index: 1723, label: 3
index: 1724, label: 242
index: 1725, label: 3
index: 1726, label: 901
index: 1727, label: 3
index: 1728, label: 377
index: 1729, label: 99
index: 1730, label: 309
index: 1731, label: 417
index: 1732, label: 417
index: 1733, label: 3
index: 1734, label: 665
index: 1735, la

index: 3192, label: 103
index: 3193, label: 103
index: 3194, label: 43
index: 3195, label: 349
index: 3196, label: 349
index: 3197, label: 156
index: 3198, label: 3
index: 3199, label: 904
index: 3200, label: 3
index: 3201, label: 3
index: 3202, label: 3
index: 3203, label: 274
index: 3204, label: 3
index: 3205, label: 3
index: 3206, label: 3
index: 3207, label: 3
index: 3208, label: 38
index: 3209, label: 3
index: 3210, label: 76
index: 3211, label: 856
index: 3212, label: 3
index: 3213, label: 3
index: 3214, label: 523
index: 3215, label: 334
index: 3216, label: 274
index: 3217, label: 3
index: 3218, label: 61
index: 3219, label: 3
index: 3220, label: 119
index: 3221, label: 3
index: 3222, label: 3
index: 3223, label: 154
index: 3224, label: 3
index: 3225, label: 53
index: 3226, label: 3
index: 3227, label: 27
index: 3228, label: 27
index: 3229, label: 27
index: 3230, label: 27
index: 3231, label: 3
index: 3232, label: 3
index: 3233, label: 3
index: 3234, label: 3
index: 3235, label:

index: 4691, label: 195
index: 4692, label: 186
index: 4693, label: 186
index: 4694, label: 166
index: 4695, label: 166
index: 4696, label: 21
index: 4697, label: 21
index: 4698, label: 267
index: 4699, label: 3
index: 4700, label: 3
index: 4701, label: 3
index: 4702, label: 103
index: 4703, label: 23
index: 4704, label: 734
index: 4705, label: 3
index: 4706, label: 3
index: 4707, label: 40
index: 4708, label: 615
index: 4709, label: 40
index: 4710, label: 209
index: 4711, label: 468
index: 4712, label: 43
index: 4713, label: 160
index: 4714, label: 3
index: 4715, label: 370
index: 4716, label: 109
index: 4717, label: 913
index: 4718, label: 269
index: 4719, label: 82
index: 4720, label: 817
index: 4721, label: 43
index: 4722, label: 447
index: 4723, label: 3
index: 4724, label: 490
index: 4725, label: 3
index: 4726, label: 3
index: 4727, label: 3
index: 4728, label: 3
index: 4729, label: 23
index: 4730, label: 3
index: 4731, label: 207
index: 4732, label: 3
index: 4733, label: 3
index

index: 6191, label: 133
index: 6192, label: 133
index: 6193, label: 133
index: 6194, label: 47
index: 6195, label: 133
index: 6196, label: 133
index: 6197, label: 3
index: 6198, label: 3
index: 6199, label: 22
index: 6200, label: 3
index: 6201, label: 12
index: 6202, label: 38
index: 6203, label: 3
index: 6204, label: 577
index: 6205, label: 3
index: 6206, label: 62
index: 6207, label: 285
index: 6208, label: 3
index: 6209, label: 3
index: 6210, label: 3
index: 6211, label: 143
index: 6212, label: 3
index: 6213, label: 3
index: 6214, label: 3
index: 6215, label: 480
index: 6216, label: 3
index: 6217, label: 414
index: 6218, label: 23
index: 6219, label: 468
index: 6220, label: 23
index: 6221, label: 23
index: 6222, label: 3
index: 6223, label: 870
index: 6224, label: 3
index: 6225, label: 3
index: 6226, label: 3
index: 6227, label: 3
index: 6228, label: 598
index: 6229, label: 3
index: 6230, label: 293
index: 6231, label: 3
index: 6232, label: 3
index: 6233, label: 3
index: 6234, label

...

index: 25684, label: 413
index: 25685, label: 3
index: 25686, label: 343
index: 25687, label: 34
index: 25688, label: 331
index: 25689, label: 625
index: 25690, label: 198
index: 25691, label: 106
index: 25692, label: 198
index: 25693, label: 829
index: 25694, label: 83
index: 25695, label: 485
index: 25696, label: 251
index: 25697, label: 291
index: 25698, label: 291
index: 25699, label: 291
index: 25700, label: 291
index: 25701, label: 659
index: 25702, label: 251
index: 25703, label: 659
index: 25704, label: 659
index: 25705, label: 931
index: 25706, label: 931
index: 25707, label: 3
index: 25708, label: 3
index: 25709, label: 586
index: 25710, label: 270
index: 25711, label: 364
index: 25712, label: 364
index: 25713, label: 364
index: 25714, label: 892
index: 25715, label: 892
index: 25716, label: 892
index: 25717, label: 892
index: 25718, label: 892
index: 25719, label: 892
index: 25720, label: 892
index: 25721, label: 892
index: 25722, label: 892
index: 25723, label: 933
index: 2

index: 27184, label: 3
index: 27185, label: 3
index: 27186, label: 3
index: 27187, label: 701
index: 27188, label: 170
index: 27189, label: 3
index: 27190, label: 3
index: 27191, label: 3
index: 27192, label: 3
index: 27193, label: 3
index: 27194, label: 3
index: 27195, label: 707
index: 27196, label: 3
index: 27197, label: 3
index: 27198, label: 67
index: 27199, label: 2
index: 27200, label: 140
index: 27201, label: 3
index: 27202, label: 687
index: 27203, label: 3
index: 27204, label: 475
index: 27205, label: 930
index: 27206, label: 3
index: 27207, label: 3
index: 27208, label: 27
index: 27209, label: 301
index: 27210, label: 24
index: 27211, label: 480
index: 27212, label: 614
index: 27213, label: 3
index: 27214, label: 3
index: 27215, label: 3
index: 27216, label: 3
index: 27217, label: 468
index: 27218, label: 61
index: 27219, label: 3
index: 27220, label: 3
index: 27221, label: 29
index: 27222, label: 3
index: 27223, label: 80
index: 27224, label: 359
index: 27225, label: 3
inde

index: 28683, label: 3
index: 28684, label: 927
index: 28685, label: 985
index: 28686, label: 490
index: 28687, label: 362
index: 28688, label: 3
index: 28689, label: 918
index: 28690, label: 3
index: 28691, label: 133
index: 28692, label: 67
index: 28693, label: 952
index: 28694, label: 3
index: 28695, label: 358
index: 28696, label: 3
index: 28697, label: 3
index: 28698, label: 3
index: 28699, label: 3
index: 28700, label: 3
index: 28701, label: 983
index: 28702, label: 3
index: 28703, label: 818
index: 28704, label: 259
index: 28705, label: 491
index: 28706, label: 169
index: 28707, label: 143
index: 28708, label: 3
index: 28709, label: 550
index: 28710, label: 480
index: 28711, label: 551
index: 28712, label: 303
index: 28713, label: 69
index: 28714, label: 3
index: 28715, label: 988
index: 28716, label: 936
index: 28717, label: 3
index: 28718, label: 333
index: 28719, label: 169
index: 28720, label: 730
index: 28721, label: 176
index: 28722, label: 12
index: 28723, label: 246
inde

index: 30183, label: 3
index: 30184, label: 3
index: 30185, label: 3
index: 30186, label: 3
index: 30187, label: 3
index: 30188, label: 3
index: 30189, label: 307
index: 30190, label: 3
index: 30191, label: 3
index: 30192, label: 3
index: 30193, label: 3
index: 30194, label: 3
index: 30195, label: 3
index: 30196, label: 3
index: 30197, label: 3
index: 30198, label: 3
index: 30199, label: 3
index: 30200, label: 3
index: 30201, label: 3
index: 30202, label: 27
index: 30203, label: 3
index: 30204, label: 3
index: 30205, label: 91
index: 30206, label: 3
index: 30207, label: 116
index: 30208, label: 3
index: 30209, label: 3
index: 30210, label: 3
index: 30211, label: 3
index: 30212, label: 7
index: 30213, label: 3
index: 30214, label: 3
index: 30215, label: 2
index: 30216, label: 2
index: 30217, label: 7
index: 30218, label: 3
index: 30219, label: 3
index: 30220, label: 353
index: 30221, label: 3
index: 30222, label: 3
index: 30223, label: 3
index: 30224, label: 3
index: 30225, label: 3
ind

index: 31682, label: 774
index: 31683, label: 664
index: 31684, label: 560
index: 31685, label: 524
index: 31686, label: 844
index: 31687, label: 791
index: 31688, label: 620
index: 31689, label: 664
index: 31690, label: 64
index: 31691, label: 791
index: 31692, label: 285
index: 31693, label: 289
index: 31694, label: 378
index: 31695, label: 939
index: 31696, label: 227
index: 31697, label: 413
index: 31698, label: 432
index: 31699, label: 545
index: 31700, label: 271
index: 31701, label: 590
index: 31702, label: 432
index: 31703, label: 545
index: 31704, label: 939
index: 31705, label: 939
index: 31706, label: 939
index: 31707, label: 334
index: 31708, label: 432
index: 31709, label: 432
index: 31710, label: 227
index: 31711, label: 420
index: 31712, label: 366
index: 31713, label: 595
index: 31714, label: 196
index: 31715, label: 590
index: 31716, label: 146
index: 31717, label: 420
index: 31718, label: 939
index: 31719, label: 939
index: 31720, label: 432
index: 31721, label: 432
i

In [206]:
kmeans.cluster_centers_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [188]:
input_tfidf = tfidf_vectorizer.transform(user_input).todense()
input_tfidf.getA()[0] 

array([0., 0., 0., ..., 0., 0., 0.])

In [112]:
#余弦相似性转换成python
def dot_product(v1, v2):
    return sum(a * b for a, b in zip(v1, v2))

def magnitude(vector):
    return math.sqrt(dot_product(vector, vector))

def similarity(v1, v2):
    return dot_product(v1, v2) / (magnitude(v1) * magnitude(v2) + .00000000001)    

In [189]:
similarity(weight[0], input_tfidf.getA()[0])

0.0

In [190]:
def cosine_similarity(x, y, norm=False):
    """ 计算两个向量x和y的余弦相似度 """
    assert len(x) == len(y), "len(x) != len(y)"
    zero_list = [0] * len(x)
    # f x == zero_list or y == zero_list:
    #    return float(1) if x == y else float(0)

    res = np.array([[x[i] * y[i], x[i] * x[i], y[i] * y[i]] for i in range(len(x))])
    cos = sum(res[:, 0]) / (np.sqrt(sum(res[:, 1])) * np.sqrt(sum(res[:, 2])))

    return 0.5 * cos + 0.5 if norm else cos  # 归一化到[0, 1]区间内

In [203]:
similar_result = []
for we in weight:
    similar_result.append(cosine_similarity(we, input_tfidf.getA()[0], norm=False))

In [204]:
similar_result

[0.0,
 0.0,
 0.0,
 0.03392726453487765,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.13572688052049464,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.01689116002387832,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.011263084409621973,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,